In [5]:
import tensorflow as tf 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten


from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
from skimage.io import imsave

import os
import numpy as np
from keras.models import load_model

In [6]:
from tools.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR
# Dataset parameters 
dataset_path = PROCESSED_DATA_DIR / 'smallerSampleDataset'
reshape_size = [224,224,3]
phishing_test_size = 0.4
num_targets = 5 

# Model parameters
input_shape = [224,224,3]
margin = 2.2
new_conv_params = [5,5,512]

# Training parameters
start_lr = 0.00002
output_dir = INTERIM_DATA_DIR / 'smallerSampleDataset'
saved_model_name = 'model' #from first training 
new_saved_model_name = 'model2'
save_interval = 200
batch_size = 16 # changed value
n_iter = 500
lr_interval = 5
# hard examples training 
num_sets = 10
iter_per_set = 8
n_iter = 30
batch_size = 16 # changed value

In [7]:
import wandb
params_dict = {
    'dataset_path': str(dataset_path),
    'reshape_size': reshape_size,
    'phishing_test_size': phishing_test_size,
    'num_targets': num_targets,
    
    'model_params': {
        'input_shape': input_shape,
        'margin': margin,
        'new_conv_params': new_conv_params
    },
    
    'training_params': {
        'start_lr': start_lr,
        'output_dir': str(output_dir),
        'saved_model_name': saved_model_name,
        'new_saved_model_name': new_saved_model_name,
        'save_interval': save_interval,
        'batch_size': batch_size,
        'n_iter': n_iter,
        'lr_interval': lr_interval
    },
    
    'hard_examples_training': {
        'num_sets': num_sets,
        'iter_per_set': iter_per_set,
        'n_iter': n_iter,
        'batch_size': batch_size
    }
}
# run = wandb.init(
#     project="Inz",
#     notes=f"VisualPhish smallerSampleDataset",
#     config={**model_config, **dataset_config, **training_config}
# )
    # run.log({"loss": loss_value})
    # run.log({"lr": start_lr})
# run.log_model(os.path.join(path, 'model.h5'), name="VP-Model")
# run.finish()


# Load dataset:
- Load training screenshots per website
- Load Phishing screenshots per website

In [9]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    
    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path / targets_list[i]
        print(target_path)
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path / file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions 
                try:
                    img = imread(target_path / file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break 
    return all_imgs,all_labels,all_file_names



In [10]:
# Read images legit (train)
data_path = dataset_path / 'trusted_list'
targets_file = open(data_path/'targets.txt', "r")
targets = targets_file.read()
imgs_num = 420
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path / 'phishing/'
targets_file = open(data_path/'targets.txt', "r")
targets = targets_file.read()
imgs_num = 160
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

# Load the same train/split in phase 1
phish_test_idx = np.load(output_dir/'test_idx.npy')
phish_train_idx = np.load(output_dir/'train_idx.npy')

X_test_phish = all_imgs_test[phish_test_idx,:]
y_test_phish = all_labels_test[phish_test_idx,:]

X_train_phish = all_imgs_test[phish_train_idx,:]
y_train_phish = all_labels_test[phish_train_idx,:]


/home/jarcin/inz/data/processed/smallerSampleDataset/trusted_list/absa
/home/jarcin/inz/data/processed/smallerSampleDataset/trusted_list/alibaba
/home/jarcin/inz/data/processed/smallerSampleDataset/trusted_list/amazon
/home/jarcin/inz/data/processed/smallerSampleDataset/trusted_list/apple
/home/jarcin/inz/data/processed/smallerSampleDataset/trusted_list/boa
/home/jarcin/inz/data/processed/smallerSampleDataset/phishing/absa
/home/jarcin/inz/data/processed/smallerSampleDataset/phishing/alibaba
/home/jarcin/inz/data/processed/smallerSampleDataset/phishing/amazon
/home/jarcin/inz/data/processed/smallerSampleDataset/phishing/apple
/home/jarcin/inz/data/processed/smallerSampleDataset/phishing/boa


In [3]:
import joblib

all_imgs_train = joblib.load(output_dir+'all_imgs_train')
all_labels_train = joblib.load(output_dir+'all_labels_train')

all_imgs_test = joblib.load(output_dir+'all_imgs_test')
all_labels_test = joblib.load(output_dir+'all_labels_test')

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

idx_test = np.load(output_dir+'test_idx.npy')
idx_train = np.load(output_dir+'train_idx.npy')
X_test_phish = all_imgs_test[idx_test,:]
y_test_phish = all_labels_test[idx_test,:]
X_train_phish = all_imgs_test[idx_train,:]
y_train_phish = all_labels_test[idx_train,:]

data_path = dataset_path + 'phishing/'

# Order and label targets

In [11]:
def order_random_array(orig_arr,y_orig_arr,targets):
    sorted_arr = np.zeros(orig_arr.shape)
    y_sorted_arr = np.zeros(y_orig_arr.shape)
    count = 0
    for i in range(0,targets):
        for j in range(0,orig_arr.shape[0]):
            if y_orig_arr[j] == i:
                sorted_arr[count,:,:,:] = orig_arr[j,:,:,:]
                y_sorted_arr[count,:] = i
                count = count + 1
    return sorted_arr,y_sorted_arr 

# Store the start and end of each target in the phishing set (used later in triplet sampling)
# Not all targets might be in the phishing set 
def start_end_each_target(num_target,labels):
    prev_target = 0
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = 0
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[count_target,1] = i-1
            count_target = count_target + 1
            start_end_each_target[count_target,0] = i
            prev_target = prev_target + 1
    start_end_each_target[num_target-1,1] = labels.shape[0]-1
    return start_end_each_target


# Store the start and end of each target in the training set (used later in triplet sampling)
def all_targets_start_end(num_target,labels):
    prev_target = labels[0]
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = labels[0]
    if not labels[0] == 0:
        start_end_each_target[0,0] = -1
        start_end_each_target[0,1] = -1
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[int(labels[i-1]),1] = int(i-1)
            #count_target = count_target + 1
            start_end_each_target[int(labels[i]),0] = int(i)
            prev_target = labels[i]
    start_end_each_target[int(labels[-1]),1] = int(labels.shape[0]-1)
    
    for i in range(1,num_target):
        if start_end_each_target[i,0] == 0:
            print(i)
            start_end_each_target[i,0] = -1
            start_end_each_target[i,1] = -1
    return start_end_each_target

labels_start_end_train_legit = all_targets_start_end(num_targets,y_train_legit)


# Finding Hard subsets

In [12]:
# Find a query set for each target
def find_fixed_set_idx(num_target):
    website_random_idx = np.zeros([num_target,])
    for i in range(0,num_target):
        class_idx_start_end = labels_start_end_train_legit[i,:]
        website_random_idx[i] = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    return website_random_idx

# Compute L2 distance between embeddings
def compute_distance_pair(layer1,layer2):
    diff = layer1 - layer2
    l2_diff = np.mean(diff**2)
    return l2_diff
    
# Compute the embeddings of the query set, the phishing training set, the training whitelist 
def predict_all_imgs(model):
    X_train_legit_last_layer = model.predict(X_train_legit,batch_size=10)
    X_train_phish_last_layer = model.predict(X_train_phish,batch_size=10)
    fixed_set_last_layer = model.predict(fixed_set,batch_size=10)
    
    return X_train_legit_last_layer,X_train_phish_last_layer,fixed_set_last_layer

# Compute distance between the query set and all training examples 
def compute_all_distances(fixed_set,train_legit,train_phish):
    train_size = train_legit.shape[0] + train_phish.shape[0]
    X_all_train = np.concatenate((train_legit,train_phish))
    pairwise_distance = np.zeros([fixed_set.shape[0],train_size])
    for i in range(0,fixed_set.shape[0]):
        pair1 = fixed_set[i,:]
        for j in range(0,train_size):
            pair2 = X_all_train[j,:]
            l2_diff = compute_distance_pair(pair1,pair2)
            pairwise_distance[i,j] = l2_diff
    return pairwise_distance

# Get index of false positives (different-website examples with small distance) of one query image
def find_n_false_positives(distances,n,test_label):
    count = 0
    X_false_pos_idx = np.zeros([n,])
    idx_min = np.argsort(distances)
    for i in range(0,distances.shape[0]):
        next_min_idx = idx_min[i]
        n_label = y_train[next_min_idx]
        #false positives (have close distance even if they are from differenet category)
        if not (test_label == n_label):
            X_false_pos_idx[count] = next_min_idx
            count = count + 1
            if count == n:
                break 
    while count < n:
        idx_min[count] = -1
        count = count + 1
    return X_false_pos_idx

# Get index of false negatives (same-website examples with large distance) of one query image
def find_n_false_negatives(distances,n,test_label):
    count = 0     
    X_false_neg_idx = np.zeros([n,])
    idx_max = np.argsort(distances)[::-1]
    for i in range(0,distances.shape[0]):
        next_max_idx = idx_max[i]
        n_label = y_train[next_max_idx]
        #false negatives (have large distance although they are in the same category )
        if test_label == n_label:
            X_false_neg_idx[count] = next_max_idx
            count = count + 1
            if count == n:
                break
    while count < n:
        idx_max[count] = -1
        count = count + 1
    return X_false_neg_idx

# Get the idx of false positives and false negtaives for all query examples
def find_index_for_all_set(distances,n):
    all_idx = np.zeros([distances.shape[0],2,n])
    for i in range(0,distances.shape[0]):
        distance_i = distances[i,:]
        all_idx[i,0,:] = find_n_false_positives(distance_i,n,i)
        all_idx[i,1,:] = find_n_false_negatives(distance_i,n,i)
    return all_idx

# Form the new training set based on the hard examples indices of all query images
def find_next_training_set(all_idx,n):
    global X_train_new, y_train_new
    all_idx = all_idx.astype(int)
    count = 0
    for i in range(all_idx.shape[0]):
        for j in range(0,n):
            if not all_idx[i,0,j] == -1:
                X_train_new[count,:,:,:] = X_train[all_idx[i,0,j],:,:,:]
                y_train_new[count,:] = y_train[all_idx[i,0,j]]
                count = count +1 
        for j in range(0,n):
            if not all_idx[i,1,j] == -1:
                X_train_new[count,:,:,:] = X_train[all_idx[i,1,j],:,:,:]
                y_train_new[count,:] = y_train[all_idx[i,1,j]]
                count = count +1 
    X_train_new = X_train_new[0:count,:]
    y_train_new = y_train_new[0:count,:]
    return X_train_new,y_train_new

# Main function for subset sampling
# Steps:
    # Predict all images
    # Find pairwise distances between query and training set
    # Find indices of hard positive and negative examples
    # Find new training set
    # Order training set by targets
def find_main_train(model,fixed_set,targets):
    X_train_legit_last_layer,X_train_phish_last_layer,fixed_set_last_layer = predict_all_imgs(model)
    pairwise_distance = compute_all_distances(fixed_set_last_layer,X_train_legit_last_layer,X_train_phish_last_layer)
    n = 1
    all_idx = find_index_for_all_set(pairwise_distance,n)
    X_train_new,y_train_new = find_next_training_set(all_idx,n)
    X_train_new,y_train_new = order_random_array(X_train_new,y_train_new,targets)
    labels_start_end_train = start_end_each_target(targets,y_train_new)
    return X_train_new,y_train_new,labels_start_end_train

In [13]:
# Don't sample negative image from the same category as the positive image (e.g. google and google drive)
# Create clusters of same-company websites (e.g. all microsoft websites)


targets_file = open(data_path/'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()

def get_idx_of_target(target_name,all_targets):
    for i in range(0,len(all_targets)):
        if all_targets[i] == target_name:
            found_idx = i
            return found_idx
        
#targets names of parent and sub websites
target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]

def get_associated_targets_idx(target_lists,all_targets):
    sub_target_lists_idx = []
    parents_ids = []
    for i in range(0,len(target_lists)):
        target_list = target_lists[i]
        parent_target = target_list[0]
        one_target_list = []
        parent_idx = get_idx_of_target(parent_target,all_targets)
        parents_ids.append(parent_idx)
        for child_target in target_list[1:]:
            child_idx = get_idx_of_target(child_target,all_targets)
            one_target_list.append(child_idx)
        sub_target_lists_idx.append(one_target_list)
    return parents_ids,sub_target_lists_idx 

parents_ids,sub_target_lists_idx  = get_associated_targets_idx(target_lists,all_targets)

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_ids:
        if img_label2 in sub_target_lists_idx[parents_ids.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[0]:
        if img_label2 in sub_target_lists_idx[0] or img_label2 == parents_ids[0]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[1]:
        if img_label2 in sub_target_lists_idx[1] or img_label2 == parents_ids[1]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[2]:
        if img_label2 in sub_target_lists_idx[2] or img_label2 == parents_ids[2]:
            if_same = 1
    return if_same


# Triplet Sampling

In [14]:
def pick_first_img_idx(labels_start_end,num_targets):
    random_target = -1
    while (random_target == -1):
        random_target = np.random.randint(low = 0,high = num_targets)
        if labels_start_end[random_target,0] == -1:
            random_target = -1
    return random_target

def pick_pos_img_idx(img_label):
    class_idx_start_end = labels_start_end_train[img_label,:]
    same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    img = X_train_new[same_idx,:]
    return img

def pick_neg_img(anchor_idx,num_targets):
    if anchor_idx == 0:
        targets = np.arange(1,num_targets)
    elif anchor_idx == num_targets -1:
        targets = np.arange(0,num_targets-1)
    else:
        targets = np.concatenate([np.arange(0,anchor_idx),np.arange(anchor_idx+1,num_targets)])
    diff_target_idx = np.random.randint(low = 0,high = num_targets-1)
    diff_target = targets[diff_target_idx]
    
    class_idx_start_end = labels_start_end_train[diff_target,:]
    idx_from_diff_target = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    img = X_train_new[idx_from_diff_target,:]
    
    return img,diff_target

#Sample batch 
def get_batch(batch_size,train_fixed_set,num_targets):
   
    # initialize 3 empty arrays for the input image batch
    h = X_train_legit.shape[1]
    w = X_train_legit.shape[2]
    triple=[np.zeros((batch_size, h, w,3)) for i in range(3)]

    for i in range(0,batch_size):
        img_idx_pair1 = pick_first_img_idx(labels_start_end_train,num_targets)
        triple[0][i,:,:,:] = train_fixed_set[img_idx_pair1,:]
        img_label = img_idx_pair1
        
        #get image for the second: positive
        triple[1][i,:,:,:] = pick_pos_img_idx(img_label)
            
        #get image for the thrid: negative from legit
        img_neg,label_neg = pick_neg_img(img_label,num_targets)
        while check_if_same_category(img_label,label_neg) == 1:
            img_neg,label_neg = pick_neg_img(img_label,num_targets)

        triple[2][i,:,:,:] = img_neg
          
    return triple

# Load Model

In [15]:

def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value

full_model = load_model(output_dir/f'{saved_model_name}.h5', custom_objects={'loss': loss})

def custom_loss(margin):
    def loss(y_true,y_pred):
        loss_value = K.maximum(y_true, margin + y_pred)
        loss_value = K.mean(loss_value,axis=0)
        return loss_value
    return loss


from keras import optimizers
optimizer = optimizers.Adam(lr = start_lr)
full_model.compile(loss=custom_loss(margin),optimizer=optimizer)

2025-01-21 00:26:52.094078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-21 00:26:52.121652: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-21 00:26:52.121704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-21 00:26:52.122700: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

# Training

In [16]:
def save_keras_model(model):
    model.save(output_dir/f'{new_saved_model_name}.h5')
    print("Saved model to disk")

In [18]:
n=1 #number of wrong points 

#all training images
X_train = np.concatenate([X_train_legit,X_train_phish])
y_train = np.concatenate([y_train_legit,y_train_phish])

#subset training
X_train_new = np.zeros([num_targets*2*n,X_train_legit.shape[1],X_train_legit.shape[2],X_train_legit.shape[3]])
y_train_new = np.zeros([num_targets*2*n,1])

targets_train = np.zeros([batch_size,1])
tot_count = 0 

print("Starting training process!")
print("\n ------------- \n")
for k in range(0,num_sets):
    print("Starting a new set!")
    print("\n ------------- \n")
    X_train_legit = all_imgs_train
    y_train_legit = all_labels_train
    
    fixed_set_idx = find_fixed_set_idx(num_targets)
    fixed_set = X_train_legit[fixed_set_idx.astype(int),:,:,:]
    
    for j in range(0,iter_per_set):
        model = full_model.layers[3]
        X_train_new,y_train_new,labels_start_end_train = find_main_train(model,fixed_set,num_targets)
        for i in range(1, n_iter):
            tot_count = tot_count + 1
            inputs=get_batch(batch_size,fixed_set,num_targets)
            loss_iteration=full_model.train_on_batch(inputs,targets_train)
            
            print("\n ------------- \n")
            print('Iteration: '+ str(i) +'. '+ "Loss: {0}".format(loss_iteration))

            if tot_count % save_interval == 0:
                save_keras_model(full_model)

            if tot_count % lr_interval == 0:
                start_lr = 0.99*start_lr
                K.set_value(full_model.optimizer.lr, start_lr)
                # wandb.config.update({
                #     'training_params': {
                #         'start_lr': start_lr  # New learning rate value
                #     }
                # }, allow_val_change=True)

save_keras_model(full_model) 

Starting training process!

 ------------- 

Starting a new set!

 ------------- 

1/1 [==============================] - 0s 26ms/step

 ------------- 

Iteration: 1. Loss: 9.086494445800781


KeyboardInterrupt: 

# Calculate Embeddings

In [ ]:
shared_model = full_model.layers[3]

whitelist_emb = shared_model.predict(X_train_legit,batch_size=64)
np.save(output_dir+'whitelist_emb2',whitelist_emb)
np.save(output_dir+'whitelist_labels2',y_train_legit )

phishing_emb = shared_model.predict(all_imgs_test,batch_size=64)
np.save(output_dir+'phishing_emb2',phishing_emb)
np.save(output_dir+'phishing_labels2',all_labels_test )